 강사님 왈, "틀리라고 낸 문제예요!"


# 네이버 블로그 포스트 상세 페이지 수집

*참고*

https://gohighbrow.com/scraping-content-inside-an-iframe/
https://www.google.com/search?q=scrape+iframe&oq=scrape+iframe&aqs=chrome..69i57j35i39l2j0l5.1975j0j7&sourceid=chrome&ie=UTF-8

In [67]:
# module import

import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

In [6]:
base_url = 'https://search.naver.com/search.naver'

# where=post&sm=tab_jum&query=%EA%B8%B0%EC%95%84%ED%83%80%EC%9D%B4%EA%B1%B0%EC%A6%88
params = {
    'where' : 'post',
    'query' : input('검색어를 입력해 주세요. ')
}

검색어를 입력해 주세요. 기아타이거즈


*참고* : 블로그 목록 태그

![blog-lists](./images/web_11.png)

In [15]:
req = requests.get(base_url, params=params)
soup = BeautifulSoup(req.text)

# sh_blog_title _sp_each_url _sp_each_title : 공백 있지만 명확하게 하고 싶으면 `.`을 사용한다.
post_li_tags = soup.select('ul#elThumbnailResultArea>li a.sh_blog_title._sp_each_url._sp_each_title')
post_li_tags

[<a class="sh_blog_title _sp_each_url _sp_each_title" href="https://blog.naver.com/jyong2288?Redirect=Log&amp;logNo=221972724113" onclick="return goOtherCR(this, 'a=blg*i.tit&amp;r=1&amp;i=90000003_0000000000000033AE9AF991&amp;u='+urlencode(this.href))" target="_blank" title="기아 타이거즈의 해설위원이 되었습니다"><strong class="hl">기아 타이거즈</strong>의 해설위원이 되었습니다</a>,
 <a class="sh_blog_title _sp_each_url _sp_each_title" href="https://blog.naver.com/psbear?Redirect=Log&amp;logNo=221981936696" onclick="return goOtherCR(this, 'a=blg*i.tit&amp;r=2&amp;i=90000003_0000000000000033AF278C38&amp;u='+urlencode(this.href))" target="_blank" title='기아 타이거즈 맷 윌리엄스(Mattew Derrick "Matt" Williams) 감독'><strong class="hl">기아 타이거즈</strong> 맷 윌리엄스(Mattew Derrick "Matt...</a>,
 <a class="sh_blog_title _sp_each_url _sp_each_title" href="https://blog.naver.com/tmdgh2040?Redirect=Log&amp;logNo=221981265398" onclick="return goOtherCR(this, 'a=blg*i.tit&amp;r=3&amp;i=90000003_0000000000000033AF1D4DF6&amp;u='+urlencode(this.href

In [12]:
# 첫 블로그 페이지에 들어가서 상세 페이지를 가져 온다.

post_a_tag = post_li_tags[0]
post_req = requests.get(post_a_tag['href'])
post_soup = BeautifulSoup(post_req.text)

## iFrame의 이해



![blog-frame](./images/web_12.png)


출처 : https://blog.naver.com/pkijj04/221975138084

In [13]:
# None
post_soup.find('div', {'id':'postListBody'})

In [14]:
# BeautifulSoup 객체로 파싱한 html 문서 확인
post_soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="ko">
<head>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="-1" http-equiv="Expires"/>
<meta content="noindex,follow" name="robots"/>
<meta content="always" name="referrer"/>
<meta content="text/html;charset=utf-8" http-equiv="content-type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<link href="/favicon.ico?2" rel="shortcut icon" type="image/x-icon"/>
<link href="https://rss.blog.naver.com/jyong2288.xml" rel="alternate" title="RSS feed for jyong2288 Blog" type="application/rss+xml"/>
<link href="https://blog.naver.com/NBlogWlwLayout.nhn?blogId=jyong2288" rel="wlwmanifest" type="application/wlwmanifest+xml"/>
<title>청연한방병원 병원장 김지용입니다. : 네이버 블로그</title>
</head>
<script charset="UTF-8" src="https://ssl.pstatic.net/t.static.blog/mylog/versioning/Frameset-657805386_https.js" type="text/javascript"></script>
<script charset

 html 문서를 확인하니까 iframe이 있다. 즉, 우리가 찾는 게, `iframe` 안에 있다.
 
![blog-iframe](./images/web_13.png)

### 왜?

requests를 보내면 response가 오는 건 만고 불변의 진리다.


- requests를 보내면, response로 iframe이 온다.??????????????????????????????????????????????????????
- requests를 잘못 날린 거다.
- iframe 보니까, `src`가 있다.
- 그 안의 주소로 가보자.

> src : "https://blog.naver.com/PostView.nhn?blogId=pkijj04&logNo=221975138084&from=search&redirect=Log&widgetTypeCall=true&topReferer=https%3A%2F%2Fsearch.naver.com%2Fsearch.naver%3Fwhere%3Dpost%26sm%3Dtab_jum%26query%3D%25EA%25B8%25B0%25EC%2595%2584%25ED%2583%2580%25EC%259D%25B4%25EA%25B1%25B0%25EC%25A6%2588&directAccess=false"



 똑같은 화면이 뜬다.
 
 그러면 거기로 요청을 보내보면 어떻게 되나?

In [16]:
# 성공!

i_url = "https://blog.naver.com/PostView.nhn?blogId=pkijj04&logNo=221975138084&from=search&redirect=Log&widgetTypeCall=true&topReferer=https%3A%2F%2Fsearch.naver.com%2Fsearch.naver%3Fwhere%3Dpost%26sm%3Dtab_jum%26query%3D%25EA%25B8%25B0%25EC%2595%2584%25ED%2583%2580%25EC%259D%25B4%25EA%25B1%25B0%25EC%25A6%2588&directAccess=false"
i_req = requests.get(i_url)
i_soup = BeautifulSoup(i_req.text)
i_soup.find('div', {'id':'postListBody'})

<div id="postListBody">
<div class="post _post_wrap _param(1)" data-post-editor-version="4" id="post_1">
<div class="post-back">
<table cellpadding="0" cellspacing="0" class="post-head"><tr><td class="htl" nowrap="nowrap"></td><td class="htc"></td><td class="htr" nowrap="nowrap"></td></tr></table>
<table cellpadding="0" cellspacing="0" class="post-body" id="printPost1" role="presentation">
<tr>
<td class="bcl" nowrap="nowrap"></td>
<td class="bcc">
<div class="wrap_rabbit pcol2 _param(1) _postViewArea221975138084" id="post-view221975138084">
<!-- Rabbit HTML --><div class="se-viewer se-theme-default" lang="ko-KR">
<!-- SE_DOC_HEADER_START -->
<div class="se-component se-documentTitle se-l-default" id="SE-c7672208-f193-48e5-ad3f-fadd8d731f8d">
<div class="se-component-content">
<div class="se-section se-section-documentTitle se-l-default se-section-align-left">
<!-- -->
<div class="blog2_series">
<a class="pcol2" href="/PostList.nhn?blogId=pkijj04&amp;categoryNo=31&amp;from=postList" on

request 한 번 보냈는데, 웹 브라우저가 하는 일은 매우 많다.
 

 툭수한 태그들이 있다. 예컨대, img 등. 태그에 있는 주소들이 있는데, 그런 걸 볼 때마다 브라우저가 다시 요청을 날린다는 거다. 

### iframe이란?


브라우저는 frame 단위로 이루어져 있다. 하나의 request는 일반적으로 하나의 frame을 만듦. 브라우저는 html을 읽어서 적절히 요청을 여러 번 보낸다.

![iframe-lecture](./images/web_15.png)
 
* iframe이란, 위에서 말한 특수한 태그의 일종이다. 
* 태그 안에 있는 주소로 접속해서 태그로서 프레임을 삽입하는 게 iframe이다. 

 한 번 요청을 보낼 때,(예컨대 우리가 그 블로그에 접속할 때) response는 iframe만 받아 온다.  request의 응답으로 iframe의 소스를 읽어서 요청을 **한 번 더** 보낸다. 처음 받는 건 iframe 하나만 받아 오고, 그 안에 있는 src의 주소로 요청을 또 보내는 것. 그런데 우리는 요청을 *한 번* 보낸다. 그래서 얘네들은 요청을 보내고, src를 읽어서 눈에 브라우저로는 보이는데, 크롤링해오려면 찾아올 수가 없다.
  
### 그러니까, 우리도 브라우저처럼, 요청을 따라서 날려줘야 한다.


<연습문제 5>

 네이버 주식 삼성전자 공시 리스트 가져오기

In [57]:
import pandas as pd

In [63]:
base_url = "https://finance.naver.com"
sise_url = f"{base_url}/item/sise_day.nhn"

params = {
    'code' : '005930'
}

req = requests.get(sise_url, params=params)
soup = BeautifulSoup(req.text)
trs = soup.find_all('tr')

spans = soup.select('span.tah')

dates = []
closing_prices = []
changes = []
opening_prices = []
highest_prices = []
lowest_prices = []
volumes = []

for idx, span in enumerate(spans):
    if idx % 7 == 0:
        dates.append(span.get_text(strip=True))
    elif idx % 7 == 1:
        closing_prices.append(span.get_text(strip=True))
    elif idx % 7 == 2:
        if 'red' in span['class'][-1]:
            changes.append(span.get_text(strip=True) + " 상승")
        else:
            changes.append(span.get_text(strip=True) + " 하락")
    elif idx % 7 == 3:
        opening_prices.append(span.get_text(strip=True))
    elif idx % 7 == 4:
        highest_prices.append(span.get_text(strip=True))
    elif idx % 7 == 5:
        lowest_prices.append(span.get_text(strip=True))
    else:
        volumes.append(span.get_text(strip=True))

In [64]:
pd.DataFrame(list(zip(dates, closing_prices, changes, opening_prices, highest_prices, lowest_prices, volumes)),
            columns = ['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량'])

,날짜,종가,전일비,시가,고가,저가,거래량
0,2020.06.02,"51,400",200 상승,"51,000","51,500","50,800","13,754,143"
1,2020.06.01,"51,200",500 상승,"50,800","51,200","50,600","16,949,183"
2,2020.05.29,"50,700",300 상승,"50,000","50,700","49,700","27,596,961"
3,2020.05.28,"50,400",500 상승,"51,100","51,200","49,900","31,309,318"
4,2020.05.27,"49,900",650 상승,"48,950","50,000","48,800","19,548,479"
5,2020.05.26,"49,250",400 상승,"48,700","49,450","48,600","15,127,490"
6,2020.05.25,"48,850",100 상승,"48,750","48,900","48,450","14,337,913"
7,2020.05.22,"48,750","1,200 하락","49,600","49,800","48,600","19,706,284"
8,2020.05.21,"49,950",50 하락,"50,300","50,400","49,850","14,949,266"
9,2020.05.20,"50,000",300 하락,"50,000","50,200","49,800","14,896,899"


## 강사님 풀이

In [84]:
base_url = "https://finance.naver.com/item/sise_day.nhn"

params = {
    'code' : '005930',
    'page' : 1
}

In [85]:
req = requests.get(base_url, params=params)

soup = BeautifulSoup(req.text)

tr_tags = soup.select('table.type2 tr') # 테이블의 tr 선택
table_header, table_row = tr_tags[0], tr_tags[1:]

In [86]:
table_header

<tr>
<th>날짜</th>
<th>종가</th>
<th>전일비</th>
<th>시가</th>
<th>고가</th>
<th>저가</th>
<th>거래량</th>
</tr>

In [89]:
table_row[:3]

[<tr>
 <td colspan="7" height="8"></td>
 </tr>,
 <tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
 <td align="center"><span class="tah p10 gray03">2020.06.02</span></td>
 <td class="num"><span class="tah p11">51,400</span></td>
 <td class="num">
 <img alt="상승" height="6" src="https://ssl.pstatic.net/imgstock/images/images4/ico_up.gif" style="margin-right:4px;" width="7"/><span class="tah p11 red02">
 				200
 				</span>
 </td>
 <td class="num"><span class="tah p11">51,000</span></td>
 <td class="num"><span class="tah p11">51,500</span></td>
 <td class="num"><span class="tah p11">50,800</span></td>
 <td class="num"><span class="tah p11">13,754,143</span></td>
 </tr>,
 <tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
 <td align="center"><span class="tah p10 gray03">2020.06.01</span></td>
 <td class="num"><span class="tah p11">51,200</span></td>
 <td class="num">
 <img alt="상승" height="6" src="https://ssl.pstatic.net/imgstock/images/images4/ico_up.gif" style="

In [90]:
stock_key = []
for th in table_header.find_all('th'):
    stock_key.append(th.text)
stock_key    

['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']

In [95]:
result_list = []
for row in table_row:
    td_tags = row.find_all('td')
    
    if len(td_tags) < 7:
        continue
    
    temp_dict = {}
    for idx, td in enumerate(td_tags):
        key = stock_key[idx]
        temp_dict[key] = td.get_text(strip=True)
        
    result_list.append(temp_dict)

0 <td align="center"><span class="tah p10 gray03">2020.06.02</span></td>
1 <td class="num"><span class="tah p11">51,400</span></td>
2 <td class="num">
<img alt="상승" height="6" src="https://ssl.pstatic.net/imgstock/images/images4/ico_up.gif" style="margin-right:4px;" width="7"/><span class="tah p11 red02">
				200
				</span>
</td>
3 <td class="num"><span class="tah p11">51,000</span></td>
4 <td class="num"><span class="tah p11">51,500</span></td>
5 <td class="num"><span class="tah p11">50,800</span></td>
6 <td class="num"><span class="tah p11">13,754,143</span></td>
0 <td align="center"><span class="tah p10 gray03">2020.06.01</span></td>
1 <td class="num"><span class="tah p11">51,200</span></td>
2 <td class="num">
<img alt="상승" height="6" src="https://ssl.pstatic.net/imgstock/images/images4/ico_up.gif" style="margin-right:4px;" width="7"/><span class="tah p11 red02">
				500
				</span>
</td>
3 <td class="num"><span class="tah p11">50,800</span></td>
4 <td class="num"><span class="tah 

In [96]:
result_list

[{'날짜': '2020.06.02',
  '종가': '51,400',
  '전일비': '200',
  '시가': '51,000',
  '고가': '51,500',
  '저가': '50,800',
  '거래량': '13,754,143'},
 {'날짜': '2020.06.01',
  '종가': '51,200',
  '전일비': '500',
  '시가': '50,800',
  '고가': '51,200',
  '저가': '50,600',
  '거래량': '16,949,183'},
 {'날짜': '2020.05.29',
  '종가': '50,700',
  '전일비': '300',
  '시가': '50,000',
  '고가': '50,700',
  '저가': '49,700',
  '거래량': '27,596,961'},
 {'날짜': '2020.05.28',
  '종가': '50,400',
  '전일비': '500',
  '시가': '51,100',
  '고가': '51,200',
  '저가': '49,900',
  '거래량': '31,309,318'},
 {'날짜': '2020.05.27',
  '종가': '49,900',
  '전일비': '650',
  '시가': '48,950',
  '고가': '50,000',
  '저가': '48,800',
  '거래량': '19,548,479'},
 {'날짜': '2020.05.26',
  '종가': '49,250',
  '전일비': '400',
  '시가': '48,700',
  '고가': '49,450',
  '저가': '48,600',
  '거래량': '15,127,490'},
 {'날짜': '2020.05.25',
  '종가': '48,850',
  '전일비': '100',
  '시가': '48,750',
  '고가': '48,900',
  '저가': '48,450',
  '거래량': '14,337,913'},
 {'날짜': '2020.05.22',
  '종가': '48,750',
  '전일비': '1,200',
  '시

# 강사님 풀이와 결합해서 최종적으로

In [104]:
base_url = "https://finance.naver.com/item/sise_day.nhn?code=005930"

wd = webdriver.Chrome('./chromedriver.exe')
wd.get(base_url)
last_page = wd.find_element_by_css_selector('td.pgRR>a')
last_page.click()

html = wd.page_source

wd.implicitly_wait(5)
wd.quit()

soup = BeautifulSoup(html)
for s in soup.select('td.on>a'):
    last_page = int(s.get_text())


results = []
for i in range(int(last_page)):
    url = f"{base_url}&page={i+1}"
    req = requests.get(url)
    
    if i % 5 == 0:
        print(f"{i+1}page, {req.status_code}")
    
    soup = BeautifulSoup(req.text)
    
    trs = soup.select('table.type2 tr') 
    
    table_header, table_body = trs[0], trs[1:]
    
    header = [th.get_text() for th in table_header.find_all('th')]    
    
    for body in table_body:
        tds = body.find_all('td')
        
        if len(tds) > 7:
            continue
        
        temp_dict = {}
        
        for idx, td in enumerate(tds):
            key = header[idx]
            temp_dict[key] = val
        
        results.append(temp_dict)
    
#     df = pd.DataFrame(results.values(), columns = result.keys())
    break

1page, 200


AttributeError: 'list' object has no attribute 'values'

In [105]:
results

[{'날짜': <span class="tah p11">14,896,899</span>},
 {'날짜': <span class="tah p11">14,896,899</span>,
  '종가': <span class="tah p11">14,896,899</span>,
  '전일비': <span class="tah p11">14,896,899</span>,
  '시가': <span class="tah p11">14,896,899</span>,
  '고가': <span class="tah p11">14,896,899</span>,
  '저가': <span class="tah p11">14,896,899</span>,
  '거래량': <span class="tah p11">14,896,899</span>},
 {'날짜': <span class="tah p11">14,896,899</span>,
  '종가': <span class="tah p11">14,896,899</span>,
  '전일비': <span class="tah p11">14,896,899</span>,
  '시가': <span class="tah p11">14,896,899</span>,
  '고가': <span class="tah p11">14,896,899</span>,
  '저가': <span class="tah p11">14,896,899</span>,
  '거래량': <span class="tah p11">14,896,899</span>},
 {'날짜': <span class="tah p11">14,896,899</span>,
  '종가': <span class="tah p11">14,896,899</span>,
  '전일비': <span class="tah p11">14,896,899</span>,
  '시가': <span class="tah p11">14,896,899</span>,
  '고가': <span class="tah p11">14,896,899</span>,
  '저가': <spa